### Instalação do XGBoost

1. Vá em File -> New -> Terminal e digite o seguinte comando:

> conda install -c conda-forge xgboost


2. Depois, execute o código abaixo para ver se a instalação funcionou corretamente

In [1]:
import xgboost

ModuleNotFoundError: No module named 'xgboost'

### Versões dos pacotes

1. Confira também se os pacotes abaixo estão nas seguintes versões:

> Python >= 3.7

> numpy >= 1.19.1

> SciPy >= 1.5.2

> Scikit-Learn >= 0.23.2

> XGBoost >= 1.2.0

In [ ]:
import platform; print(platform.platform())
import sys; print("Python", sys.version)
import numpy; print("NumPy", numpy.__version__)
import scipy; print("SciPy", scipy.__version__)
import sklearn; print("Scikit-Learn", sklearn.__version__)
import xgboost; print("XGBoost", xgboost.__version__)

# Criando algoritmos baselines

Antes de analisarmos o algoritmo de Árvore de Decisão, vamos criar aqui algoritmos baseline e compara-los minimanente com o XGBoost, tanto para regressão quanto para classificação. Isso será importante para podermos comparar com os resultados de quando fizermos os ajustes nos hiperparâmetros do XGBoost

### Regressão

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_bikes = pd.read_csv('bases/bike_rentals_cleaned.csv', sep=',')
df_bikes.head()

In [ ]:
#separando os dados em features e labels
X = df_bikes.iloc[:,:-1]
y = df_bikes.iloc[:,-1]

In [ ]:
#vamos treinar uma regressão linear e comparar com o XGBoost depois
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

y_pred = lin_reg.predict(X_test)

from sklearn.metrics import mean_squared_error
import numpy as np

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print("RMSE: %0.2f" % (rmse))

In [ ]:
#podemos usar o método describe para tentar qualificar o RMSE
df_bikes['cnt'].describe()

In [ ]:
from xgboost import XGBRegressor
xg_reg = XGBRegressor()

xg_reg.fit(X_train, y_train)
y_pred = xg_reg.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print("RMSE: %0.2f" % (rmse))

### Cross-Validation

Realizar apenas um teste não é confiável, visto que se você fizer uma divisão dos dados diferente, você obterá diferentes resultados. Para tratar isso, usamos cross-validation, cujo processo já foi explicado na aula de SVM.

In [ ]:
#para regressão linear
from sklearn.model_selection import cross_val_score
model = LinearRegression()
scores = cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=10)
rmse = np.sqrt(-scores)
print('Reg rmse:', np.round(rmse, 2))
print('RMSE mean: %0.2f' % (rmse.mean()))

In [ ]:
# para XGBoost
model = XGBRegressor()
scores = cross_val_score(model, X, y, scoring='neg_mean_squared_error', cv=10)
rmse = np.sqrt(-scores)
print('Reg rmse:', np.round(rmse, 2))
print('RMSE mean: %0.2f' % (rmse.mean()))

### Classificação

In [ ]:
df_census = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', header=None)
df_census.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation',
                  'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 
                   'income']
df_census.head()

In [ ]:
df_census.info()

Quando olhamos para o dataset, vemos a coluna 'education' e 'education_num'. A última é a versão numérica da primeira. Assim, podemos dropar 'education'. 

In [ ]:
df_census = df_census.drop(['education'], axis=1)

Todas as colunas precisam ser transformadas em colunas numéricas. O método *get_dumies* do pandas pega valores únicos não-numéricos de cada coluna e os converte em colunas, com 1 indicando presença e 0 ausência. Veja o exemplo 

In [ ]:
#exemplo
pd.get_dummies(df_census['marital-status'])

In [ ]:
df_census = pd.get_dummies(df_census)
df_census.head()

O objetivo (coluna target) é determinar se uma pessoa ganha mais que 50k ou não. Depois do pd.get_dummies, duas colunas, df_census['income_<=50k'] e df_census['income_>50k] são usadas para determinar se uma pessoa ganha 50k. Visto que ambas representam a mesma coisa, vamos deletar a primeira

In [ ]:
df_census = df_census.drop('income_ <=50K', axis=1)

In [ ]:
#features e labels
X = df_census.iloc[:,:-1]
y = df_census.iloc[:,-1]

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
#uma função para executar cross-validation
def cross_val(classifier, num_splits=10):
    model = classifier
    scores = cross_val_score(model, X, y, cv=num_splits)
    print('Accuracy:', np.round(scores, 2))
    print('Accuracy mean: %0.2f' % (scores.mean()))

In [ ]:
cross_val(KNeighborsClassifier())

In [ ]:
from xgboost import XGBClassifier

In [ ]:
cross_val(XGBClassifier(n_estimators=5))

# Decision Trees

Vamos começar executando decision trees para comparar os resultados com o dataset df_census e depois entender como usar GridSearch para encontrar os melhores parâmetros para nossa árvore de decisão

In [ ]:
X = df_census.iloc[:,:-1]
y = df_census.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
clf = DecisionTreeClassifier(random_state=2)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_pred, y_test)

Agora vamos treinar uma árvore de regressão usando GridSearch

In [ ]:
df_bikes = pd.read_csv('bases/bike_rentals_cleaned.csv')

X_bikes = df_bikes.iloc[:,:-1]
y_bikes = df_bikes.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X_bikes, y_bikes, random_state=2)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score

In [ ]:
reg = DecisionTreeRegressor(random_state=2)
scores = cross_val_score(reg, X_bikes, y_bikes, scoring='neg_mean_squared_error', cv=5)
rmse = np.sqrt(-scores)
print('RMSE mean: %0.2f' % (rmse.mean()))

Podemos checar se o modelo esta overfitando ou não. para isso, podemos predizer o proprio conjunto de treino e verificar o RMSE

In [ ]:
reg = DecisionTreeRegressor()
reg.fit(X_train, y_train)
y_pred = reg.predict(X_train)
from sklearn.metrics import mean_squared_error
reg_mse = mean_squared_error(y_train, y_pred)
reg_rmse = np.sqrt(reg_mse)
reg_rmse

Como suspeitamos, o modelo está overfitando. Assim, precisamos usar o GridSearchCV para determinar o melhor valor dos hiperparâmetos.

Vamos começar por *max_depth*

In [ ]:
from sklearn.model_selection import GridSearchCV
params = {'max_depth':[None,2,3,4,6,8,10,20]}
reg = DecisionTreeRegressor(random_state=2)
grid_reg = GridSearchCV(reg, params, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)


grid_reg.fit(X_train, y_train)

best_params = grid_reg.best_params_
print("Best params:", best_params)

In [ ]:
#e agora verificamos se com esse valor, o RMSE diminui
best_model = grid_reg.best_estimator_
y_pred = best_model.predict(X_test)
rmse_test = mean_squared_error(y_test, y_pred)**0.5

print('Test score: {:.3f}'.format(rmse_test))

Para testar com outras features, vamos criar uma função para facilitar nosso trabalho:

In [ ]:
def grid_search(params, reg=DecisionTreeRegressor(random_state=2)):

    grid_reg = GridSearchCV(reg, params, scoring='neg_mean_squared_error', cv=5, n_jobs=-1)
    
    grid_reg.fit(X_train, y_train)

    best_params = grid_reg.best_params_

    print("Best params:", best_params)
    
    best_score = np.sqrt(-grid_reg.best_score_)

    print("Training score: {:.3f}".format(best_score))

    y_pred = grid_reg.predict(X_test)

    rmse_test = mean_squared_error(y_test, y_pred)**0.5

    print('Test score: {:.3f}'.format(rmse_test))

In [ ]:
#vamos testar para a feature min_samples_leaf
grid_search(params={'min_samples_leaf':[1,2,4,6,8,10,20,30]})

Vamos ver o que acontece quando usamos as duas features no gridsearch

In [ ]:
grid_search(params={'max_depth':[None,2,3,4,6,8,10,20],'min_samples_leaf':[1,2,4,6,8,10,20,30]})

Surpreendentemente, o score aumentou. Precisamos sempre colocar vários parÂmetros juntos para executar o GridSearch.

vamos iniciar min_samples_leaf em 3 e max_depth a partir de 6 e ver o que acontece:

In [ ]:
grid_search(params={'max_depth':[5,6,7,8,9],'min_samples_leaf':[3,5,7,9]})

Vimos que o score melhorou, mas ainda temos um longo caminho pela frente. 

# Bagging

### Random Forest Classifiers

Vamos começar usando o dataset df_census e comparar o Random Forest com Árvores de Decisão e XGBoost

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
X_census = df_census.iloc[:,:-1]
y_census = df_census.iloc[:,-1]

In [ ]:
rf = RandomForestClassifier(n_estimators=10, random_state=2, n_jobs=-1)

#n_estimators = número de árvores
#n_jobs = quando igual a -1, usa todos os processadores para execução em paralelo

scores = cross_val_score(rf, X_census, y_census, cv=5)
print('Accuracy:', np.round(scores, 3))
print('Accuracy mean: %0.3f' % (scores.mean()))

A versão default do Random Forest forneceu um resultado melhor que o das Árvores de Decisão (81%), mas não tão bom quanto o XGBoost (86%). 

Mas aqui, a grande questão é: porque RF foi melhor que uma árvore de decisão individual?

A resposta curta é: **bagging**. Com 10 árvores (n_estimator=10), cada predição é baseada em 10 árvores de decisão ao invés de em apenas uma. O Bootstrap age também, fornecendo diversidade e reduzindo a variância.

Por padrão, Random Forest Classifiers selecionam a partir da raiz quadrada do número total de features ao fazer um split. Então, se existem 100 features (Colunas), cada árvore de decisão vai considerar apenas 10 features quando fazer o split. Assim, duas árvores com amostras duplicadas podem fornecer predições muito diferentes devido aos diferentes splits. Essa é outra maneira de reduzir variância.  

### Random Forest Regressor

Numa Random Forest Regressor, os exemplos são amostrados com reposição, da mesma maneira que num Random Forest Classifier, mas o número máximo de features é o número total de features ao invés da raiz quadrada. Isto é devido a experimentos. Veja [aqui](https://orbi.uliege.be/bitstream/2268/9357/1/geurts-mlj-advance.pdf)

Além disso, a predição final é feita levando em consideração a média das predições de todas as árvores ao invés de regras baseadas em voto majoritário. 

In [ ]:
X_bikes = df_bikes.iloc[:,:-1]
y_bikes = df_bikes.iloc[:,-1]

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=10, random_state=2, n_jobs=-1)
scores = cross_val_score(rf, X_bikes, y_bikes, scoring='neg_mean_squared_error', cv=10)

rmse = np.sqrt(-scores)
print('RMSE:', np.round(rmse, 3))
print('RMSE mean: %0.3f' % (rmse.mean()))

Vamos agora ajustar os hiperparâmetros do Random Forest e aproveitar para introduzir uma nova maneira de fazer GridSearch, chamada de RandomizedSearchCV. A ideia é a mesma por trás do GridSearchCV, porém, como este pode levar muito tempo, RandomizedSearchCV trabalha escolhendo um número aleatório de combinações. Não foi feito para ser exaustivo, mas sim para encontrar a melhor combinação de parâmetros num tempo limitado.


Vamos voltar a dataset df_bikes

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_bikes, y_bikes, random_state=2)

In [ ]:
rf = RandomForestRegressor(n_estimators=50, warm_start=True, n_jobs=-1, random_state=2)
scores = cross_val_score(rf, X_bikes, y_bikes, scoring='neg_mean_squared_error', cv=10)
rmse = np.sqrt(-scores)
print('RMSE:', np.round(rmse, 3))
print('RMSE mean: %0.3f' % (rmse.mean()))

Introduzimos um novo parâmetro aqui: **warm_start**.

Quando warm_star=True, adicionar mais árvores não requer começar tudo do zero. Se você mudar n_stimators de 100 para 200, Random Forest vai reusar a solução do fit() anterior, tornando a execução mais rápida. 

Agora vamos usar o RandomizedSearchCV. Para isso, vamos criar uma função que recebe os parâmetros que eu quero otimizar, a quantidade de vezes que vamos executar o algoritmo e o regressor.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error as MSE

def randomized_search_reg(params, runs=16, reg=RandomForestRegressor(random_state=2, n_jobs=-1)):
    rand_reg = RandomizedSearchCV(reg, params, n_iter=runs, scoring='neg_mean_squared_error', 
                                  cv=10, n_jobs=-1, random_state=2)
    
    rand_reg.fit(X_train, y_train)

    best_model = rand_reg.best_estimator_

    best_params = rand_reg.best_params_

    print("Best params:", best_params)
    
    best_score = np.sqrt(-rand_reg.best_score_)

    print("Training score: {:.3f}".format(best_score))

    y_pred = best_model.predict(X_test)

    rmse_test = MSE(y_test, y_pred)**0.5
    
    print('Test set score: {:.3f}'.format(rmse_test))

In [ ]:
#agora executamos

randomized_search_reg(params={
                          'min_samples_leaf':[1,2,4,6,8,10,20,30],
                          'min_impurity_decrease':[0.0, 0.01, 0.05, 0.10, 0.15, 0.2],
                          'max_features':['auto', 0.8, 0.7, 0.6, 0.5, 0.4],
                          'max_depth':[None,4,6,8,10,12,15,20]
                         }, runs=20)

# Boosting

Para entendermos a maneira correra que o Gradient Boosting funciona, vamos usar uma Árvore de Regressão. 

In [ ]:
X_bikes = df_bikes.iloc[:,:-1]
y_bikes = df_bikes.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X_bikes, y_bikes, random_state=2)

1. Aqui, vamos usar um base learner, então configuramos max_depth=2, pois queremos aprender a partir dos erros.

In [ ]:
# Vamos treinar um Decision tree Regressor
tree_1 = DecisionTreeRegressor(max_depth=2, random_state=2)
tree_1.fit(X_train, y_train)

2. Vamos fazer predição com o conjunto de treino: ao invés de fazer predições com o conjunto de teste, no gradient boosting as predições são feitas inicialmente com o conjunto de treino. Porque? Para comparar os residuals, precisamos comparar as predições enquanto estamos na fase de treino. A fase de teste do modelo acontece no final, quando todas as árvores estiverem construídas. 

In [ ]:
y_train_pred = tree_1.predict(X_train)

3. Agora, calculamos os residuals: a diferença entre o predito e o observado

In [ ]:
y2_train = y_train - y_train_pred

**NOTA**: os residuals são definidos como y2_train por que agora eles são as labels da próxima árvore

4. Treine uma nova árvore em cima dos residuals. Isso vai, gradativamente, diminuindo os residuals

In [ ]:
tree_2 = DecisionTreeRegressor(max_depth=2, random_state=2)
tree_2.fit(X_train, y2_train)

5. Repita os passos 2 a 4. Conforme o processo continua, os residuals gradativamente vão se aproximando de 0. 

In [ ]:
y2_train_pred = tree_2.predict(X_train)
y3_train = y2_train - y2_train_pred


tree_3 = DecisionTreeRegressor(max_depth=2, random_state=2)
tree_3.fit(X_train, y3_train)

6. Agora somamos os resultados: somar os resultados requer fazer predições para cada árvore com o conjunto de teste.

In [ ]:
y1_pred = tree_1.predict(X_test)

y2_pred = tree_2.predict(X_test)

y3_pred = tree_3.predict(X_test)

y_pred = y1_pred + y2_pred + y3_pred

7. Agora, podemos calcular o MSE para obter os resultados

In [ ]:
MSE(y_test, y_pred)**0.5
#um resultado excelente para um weak learner, principalmente quando comparamos com o resultado da 
#árvore de regressão

### Gradient Boosting usando sklearn

Podemos treinar um Gradiet Boosting usando a scikit-learn para ver se chegamos no mesmo resultado

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
gbr = GradientBoostingRegressor(max_depth=2, n_estimators=3, random_state=2, learning_rate=1.0)

gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)
MSE(y_test, y_pred)**0.5


Temos um novo parâmetro: learning_rate. Para entender sua influência (e também no XGBoost), vamos treinar dois GBs mudando somente *n_estimators*:

In [ ]:
gbr = GradientBoostingRegressor(max_depth=2, n_estimators=30, random_state=2, learning_rate=1.0)
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)
MSE(y_test, y_pred)**0.5

Uma melhora significativa. Vamos testar com 300, então.

In [ ]:
gbr = GradientBoostingRegressor(max_depth=2, n_estimators=300, random_state=2, learning_rate=1.0)
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)
MSE(y_test, y_pred)**0.5

Opa! Algo de errado não está certo. Aqui vemos a importância do learning_rate. 

Basicamente, a ideia consiste em reduzir a contribuição de árvores individuais de modo que nenhuma árvore tenha muita influência ao construir o modelo, ou seja, o learning rate limita a influência de árvores individuais. Assim, se diminuirmos seu valor, veremos uma grande diferença:

In [ ]:
gbr = GradientBoostingRegressor(max_depth=2, n_estimators=300, random_state=2, learning_rate=0.1) #default sklearn
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)
MSE(y_test, y_pred)**0.5

Agora, podemos executar um RandomizedSearchCV e encontrar os melhores parâmetros (Dentro de um range específico)

In [ ]:
params={'subsample':[0.65, 0.7, 0.75],
                          'n_estimators':[300, 500, 1000],
                          'learning_rate':[0.05, 0.075, 0.1]
       }
gbr = GradientBoostingRegressor(max_depth=3, random_state=2)

rand_reg = RandomizedSearchCV(gbr, params, n_iter=10, scoring='neg_mean_squared_error', 
                              cv=5, n_jobs=-1, random_state=2)

rand_reg.fit(X_train, y_train)


best_model = rand_reg.best_estimator_
best_params = rand_reg.best_params_
print("Best params:", best_params)
best_score = np.sqrt(-rand_reg.best_score_)
print("Training score: {:.3f}".format(best_score))
y_pred = best_model.predict(X_test)
rmse_test = MSE(y_test, y_pred)**0.5
print('Test set score: {:.3f}'.format(rmse_test))

**NOTA**: usamos um parâmetro a mais: subsample. Como o nome sugere, é um subconjunto de amostras. Isso signifca que nem todas as linhas serão incluídas ao construir uma árvore. Quando seu valor é diferente de 1.0, o modelo é classificado como **Stochastic Gradient Descent**, em que *stochastic* significa alguma aleatoriedade é inerente ao modelo. 

# XGBoost

## Classification

In [ ]:
from sklearn import datasets
iris = datasets.load_iris()

In [ ]:
df = pd.DataFrame(data= np.c_[iris['data'], iris['target']],columns= iris['feature_names'] + ['target'])
df.head()

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(iris['data'], iris['target'], random_state=2)

In [ ]:
xgb = XGBClassifier(booster='gbtree', objective='multi:softprob', 
                    learning_rate=0.1, n_estimators=100, random_state=2, n_jobs=-1)
xgb.fit(X_train, y_train)
y_pred = xgb.predict(X_test)
score = accuracy_score(y_pred, y_test)
print('Score: ' + str(score))

Algumas considerações sobre os parâmetros:

1. **booster='gbtree'**: booster é o *base learner*. 'gbtree' é o gradient boosted tree

2. **objective='multi:softprob'**: função objetivo. No caso da classificação, calcula a probabilidade de cada classe e escolhe aquela com maior valor. 

## Regression

In [ ]:
from xgboost import XGBRegressor

X,y = datasets.load_diabetes(return_X_y=True)

xgb = XGBRegressor(booster='gbtree', objective='reg:squarederror', 
                    learning_rate=0.1, n_estimators=100, random_state=2, n_jobs=-1)

scores = cross_val_score(xgb, X, y, scoring='neg_mean_squared_error', cv=5)
rmse = np.sqrt(-scores)
print('RMSE:', np.round(rmse, 3))
print('RMSE mean: %0.3f' % (rmse.mean()))

1. **objective='reg:squarederror'**: é o MSE a partir do qual derivamos a função objetivo

In [ ]:
#para poder comparar o resultado, podemos verificar usar o método describe na variável y (label) e obter 
#suas principais estatísticas

In [ ]:
pd.DataFrame(y).describe()

O resultado de 63.124 é menor que 1 desvio padrão, o que é um resultado respeitável. 

# Conclusão

Agora, temos um template tanto para classificação quanto para regressão que podemos usar como ponto de partida para resolver problemas usando XGBoost.